In [1]:
import re
from io import StringIO
from datetime import datetime, timedelta

import requests
import pandas as pd


class YahooFinanceHistory:
    timeout = 2
    crumb_link = 'https://finance.yahoo.com/quote/{0}/history?p={0}'
    crumble_regex = r'CrumbStore":{"crumb":"(.*?)"}'
    quote_link = 'https://query1.finance.yahoo.com/v7/finance/download/{quote}?period1={dfrom}&period2={dto}&interval=1d&events=history&crumb={crumb}'

    def __init__(self, symbol, days_back=7):
        self.symbol = symbol
        self.session = requests.Session()
        self.dt = timedelta(days=days_back)

    def get_crumb(self):
        response = self.session.get(self.crumb_link.format(self.symbol), timeout=self.timeout)
        response.raise_for_status()
        match = re.search(self.crumble_regex, response.text)
        if not match:
            raise ValueError('Could not get crumb from Yahoo Finance')
        else:
            self.crumb = match.group(1)

    def get_quote(self):
        if not hasattr(self, 'crumb') or len(self.session.cookies) == 0:
            self.get_crumb()
        now = datetime.utcnow()
        dateto = int(now.timestamp())
        datefrom = int((now - self.dt).timestamp())
        url = self.quote_link.format(quote=self.symbol, dfrom=datefrom, dto=dateto, crumb=self.crumb)
        response = self.session.get(url)
        response.raise_for_status()
        return pd.read_csv(StringIO(response.text), parse_dates=['Date'])

In [2]:
df = YahooFinanceHistory('AAPL', days_back=30).get_quote()

In [3]:
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-03-11,175.490005,179.119995,175.350006,178.899994,178.899994,32011000
1,2019-03-12,180.000000,182.669998,179.369995,180.910004,180.910004,32467600
2,2019-03-13,182.250000,183.300003,180.919998,181.710007,181.710007,31032500
3,2019-03-14,183.899994,184.100006,182.559998,183.729996,183.729996,23579500
4,2019-03-15,184.850006,187.330002,183.740005,186.119995,186.119995,39042900
5,2019-03-18,185.800003,188.389999,185.789993,188.020004,188.020004,26219800
6,2019-03-19,188.350006,188.990005,185.919998,186.529999,186.529999,31646400
7,2019-03-20,186.229996,189.490005,184.729996,188.160004,188.160004,31035200
8,2019-03-21,190.020004,196.330002,189.809998,195.089996,195.089996,51034200
9,2019-03-22,195.339996,197.690002,190.779999,191.050003,191.050003,42407700
